In [16]:
import torch
from torchvision import transforms
from torchvision.models import resnet
from PIL import Image
import requests
from collections import namedtuple

# Load the pre-trained ResNet-50 model
model = resnet.resnet50(num_classes=1000, pretrained=True)

# Download the ImageNet class labels
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
labels = requests.get(LABELS_URL).json()

# Set the model to evaluation mode
model.eval()

# Define the transformation to be applied to your input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to match the model's expected input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess your input image
image_path = './datasets/frog_sample_1.png'
image = Image.open(image_path).convert('RGB')  # Ensure the image has 3 channels
image = transform(image).unsqueeze(0)  # Add a batch dimension

# Make predictions
with torch.no_grad():
    output = model(image)

# Get the predicted class and confidence
probabilities = torch.nn.functional.softmax(output[0], dim=0)
confidence, predicted_class = torch.max(probabilities, 0)

# Get the class name
class_name = labels[predicted_class.item()]

print(f"Predicted class: {class_name}")
print(f"Confidence: {confidence.item() * 100:.2f}%")


Predicted class: three-toed sloth
Confidence: 15.33%


In [24]:
%pip install cifar10_models

ERROR: Could not find a version that satisfies the requirement cifar10_models (from versions: none)
ERROR: No matching distribution found for cifar10_models
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torchvision import transforms, datasets
from torchvision.models import resnet
from torch.utils.data import DataLoader
import os

# # Set random seed for reproducibility
# torch.manual_seed(42)

# # Load the pre-trained ResNet-50 model
# model = resnet.resnet50(num_classes=1000, pretrained=True)

# # Set the model to evaluation mode
# model.eval()

# Define the transformation to be applied to your input image
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
])

# Download CIFAR-10 dataset
cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [3]:
# Create a DataLoader for the dataset
from tqdm import tqdm
from itertools import islice
limited_dataloader = islice(DataLoader(cifar_dataset, batch_size=1, shuffle=True), 1000)
# Loop through the images and make predictions
correct_predictions = 0
from resnet import resnet50

# Set random seed for reproducibility
torch.manual_seed(42)

# Load the pre-trained ResNet-50 model
model = resnet50(pretrained=True)

# Set the model to evaluation mode
model.eval()




for i, (image, label) in tqdm(enumerate(limited_dataloader), total=1000):
    with torch.no_grad():
        output = model(image)

    _, predicted_class = output.max(1)

    # Check if the prediction is correct
    correct_predictions += (predicted_class == label).item()

# Print the total number of correct predictions
print(f"Total correct predictions: {correct_predictions}")

100%|██████████| 1000/1000 [00:52<00:00, 19.13it/s]

Total correct predictions: 995
